In [1]:
loans = read.csv('loans.csv')

In [2]:
str(loans)

'data.frame':	9578 obs. of  14 variables:
 $ credit.policy    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ purpose          : Factor w/ 7 levels "all_other","credit_card",..: 3 2 3 3 2 2 3 1 5 3 ...
 $ int.rate         : num  0.119 0.107 0.136 0.101 0.143 ...
 $ installment      : num  829 228 367 162 103 ...
 $ log.annual.inc   : num  11.4 11.1 10.4 11.4 11.3 ...
 $ dti              : num  19.5 14.3 11.6 8.1 15 ...
 $ fico             : int  737 707 682 712 667 727 667 722 682 707 ...
 $ days.with.cr.line: num  5640 2760 4710 2700 4066 ...
 $ revol.bal        : int  28854 33623 3511 33667 4740 50807 3839 24220 69909 5630 ...
 $ revol.util       : num  52.1 76.7 25.6 73.2 39.5 51 76.8 68.6 51.1 23 ...
 $ inq.last.6mths   : int  0 0 1 1 0 0 0 0 1 1 ...
 $ delinq.2yrs      : int  0 0 0 0 1 0 0 0 0 0 ...
 $ pub.rec          : int  0 0 0 0 0 0 1 0 0 0 ...
 $ not.fully.paid   : int  0 0 0 0 0 0 1 1 0 0 ...


In [3]:
table(loans$not.fully.paid)


   0    1 
8045 1533 

In [4]:
1533 / (8045 + 1533 )

[1] 0.1600543

In [5]:
summary(loans)

 credit.policy                 purpose        int.rate       installment    
 Min.   :0.000   all_other         :2331   Min.   :0.0600   Min.   : 15.67  
 1st Qu.:1.000   credit_card       :1262   1st Qu.:0.1039   1st Qu.:163.77  
 Median :1.000   debt_consolidation:3957   Median :0.1221   Median :268.95  
 Mean   :0.805   educational       : 343   Mean   :0.1226   Mean   :319.09  
 3rd Qu.:1.000   home_improvement  : 629   3rd Qu.:0.1407   3rd Qu.:432.76  
 Max.   :1.000   major_purchase    : 437   Max.   :0.2164   Max.   :940.14  
                 small_business    : 619                                    
 log.annual.inc        dti              fico       days.with.cr.line
 Min.   : 7.548   Min.   : 0.000   Min.   :612.0   Min.   :  179    
 1st Qu.:10.558   1st Qu.: 7.213   1st Qu.:682.0   1st Qu.: 2820    
 Median :10.928   Median :12.665   Median :707.0   Median : 4140    
 Mean   :10.932   Mean   :12.607   Mean   :710.8   Mean   : 4562    
 3rd Qu.:11.290   3rd Qu.:17.950   3rd 

In [6]:
library(mice)

set.seed(144)

vars.for.imputation = setdiff(names(loans), "not.fully.paid")

imputed = complete(mice(loans[vars.for.imputation]))

loans[vars.for.imputation] = imputed

ERROR: Error in library(mice): there is no package called ‘mice’


ERROR: Error in eval(expr, envir, enclos): could not find function "complete"


ERROR: Error in eval(expr, envir, enclos): object 'imputed' not found


In [7]:
loans = read.csv('loans_imputed.csv')

In [8]:
set.seed(144)
 library(caTools)

split = sample.split(loans$not.fully.paid, SplitRatio = 0.7)

train = subset(loans, split == TRUE)

test = subset(loans, split == FALSE)

In [9]:
loansLog1 = glm(not.fully.paid ~ ., data=train, family=binomial)

In [10]:
summary(loansLog1)


Call:
glm(formula = not.fully.paid ~ ., family = binomial, data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2049  -0.6205  -0.4951  -0.3606   2.6397  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                9.187e+00  1.554e+00   5.910 3.42e-09 ***
credit.policy             -3.368e-01  1.011e-01  -3.332 0.000861 ***
purposecredit_card        -6.141e-01  1.344e-01  -4.568 4.93e-06 ***
purposedebt_consolidation -3.212e-01  9.183e-02  -3.498 0.000469 ***
purposeeducational         1.347e-01  1.753e-01   0.768 0.442201    
purposehome_improvement    1.727e-01  1.480e-01   1.167 0.243135    
purposemajor_purchase     -4.830e-01  2.009e-01  -2.404 0.016203 *  
purposesmall_business      4.120e-01  1.419e-01   2.905 0.003678 ** 
int.rate                   6.110e-01  2.085e+00   0.293 0.769446    
installment                1.275e-03  2.092e-04   6.093 1.11e-09 ***
log.annual.inc            -4.337e-01  

In [11]:
-9.317e-03*10

[1] -0.09317

In [12]:
exp(0.09317)

[1] 1.097648

In [13]:
predictTest = predict(loansLog1, type="response", newdata=test)

In [14]:
test$predicted.risk = predictTest

In [15]:
table(test$not.fully.paid, predictTest > 0.5)

   
    FALSE TRUE
  0  2400   13
  1   457    3

In [16]:
# accuracy
(2400 + 3) / (2400 + 3 + 13 + 457)

[1] 0.8364079

In [17]:
# accuracy baseline
(2400 + 13) /  (2400 + 3 + 13 + 457)

[1] 0.8398886

In [18]:
library(ROCR)
ROCRpred = prediction(predictTest, test$not.fully.paid)
as.numeric(performance(ROCRpred, "auc")@y.values)

Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess



[1] 0.6720995

In [19]:
loansLog2 = glm(not.fully.paid ~ int.rate, data=train, family=binomial)

In [20]:
summary(loansLog2)


Call:
glm(formula = not.fully.paid ~ int.rate, family = binomial, data = train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.0547  -0.6271  -0.5442  -0.4361   2.2914  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -3.6726     0.1688  -21.76   <2e-16 ***
int.rate     15.9214     1.2702   12.54   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 5896.6  on 6704  degrees of freedom
Residual deviance: 5734.8  on 6703  degrees of freedom
AIC: 5738.8

Number of Fisher Scoring iterations: 4


In [21]:
predictTest2 = predict(loansLog2, type="response", newdata=test)

In [22]:
summary(predictTest2)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.06196 0.11550 0.15080 0.15960 0.18930 0.42660 

In [23]:
table(test$not.fully.paid, predictTest2 > 0.5)

   
    FALSE
  0  2413
  1   460

In [24]:
2413+460

[1] 2873

In [25]:
ROCRpred2 = prediction(predictTest2, test$not.fully.paid)
as.numeric(performance(ROCRpred2, "auc")@y.values)

[1] 0.6239081

In [26]:
10*exp(0.06*3)

[1] 11.97217

In [31]:
test$profit = 1*exp(test$int.rate*3) - 1

test$profit[test$not.fully.paid == 1] = -1

In [30]:
summary(test$profit)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -1.000   2.858   4.111   3.535   4.980   8.895 

In [32]:
test15 = subset(test, int.rate >= 0.15)

In [33]:
summary(test15$profit)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-1.0000 -1.0000  0.5992  0.2251  0.6380  0.8895 

In [34]:
table(test15$not.fully.paid)


  0   1 
327 110 

In [35]:
110 / (110+327)

[1] 0.2517162

In [36]:
cutoff = sort(test15$predicted.risk, decreasing=FALSE)[100]

In [37]:
cutoff

[1] 0.1763305

In [38]:
test15lowrisk = subset(test15, predicted.risk <= 0.17633052990285)

In [39]:
str(test15lowrisk)

'data.frame':	100 obs. of  16 variables:
 $ credit.policy    : int  1 1 1 1 1 1 1 1 1 1 ...
 $ purpose          : Factor w/ 7 levels "all_other","credit_card",..: 7 2 3 1 3 5 2 3 2 3 ...
 $ int.rate         : num  0.15 0.153 0.158 0.159 0.156 ...
 $ installment      : num  225 444 420 246 245 ...
 $ log.annual.inc   : num  12.3 11 11.5 11.5 10.8 ...
 $ dti              : num  6.45 19.52 18.55 24.19 2.72 ...
 $ fico             : int  677 667 667 667 672 702 667 672 662 682 ...
 $ days.with.cr.line: num  6240 2701 4560 5376 3010 ...
 $ revol.bal        : int  56411 33074 34841 590 3273 4980 15977 16473 22783 87502 ...
 $ revol.util       : num  75.3 68.8 89.6 84.3 69.6 55.3 83.6 94.1 93.7 96.4 ...
 $ inq.last.6mths   : int  0 2 0 0 1 1 0 2 3 0 ...
 $ delinq.2yrs      : int  0 0 0 0 0 0 0 2 1 1 ...
 $ pub.rec          : int  0 0 0 0 0 0 0 0 0 0 ...
 $ not.fully.paid   : int  1 0 0 0 1 0 0 0 0 0 ...
 $ predicted.risk   : num  0.164 0.169 0.158 0.162 0.147 ...
 $ profit           : num  -1

In [40]:
sum(test15lowrisk$profit)

[1] 31.27825

In [41]:
table(test15lowrisk$not.fully.paid )


 0  1 
81 19 